In [30]:
import json
from tqdm import tqdm

import torch

from data import ScanDataset,ScanAugmentedDataset,MTDataset,SCAN_collate
from SymbolicOperator import SymbolicOperator

In [2]:
with open('vocab.json','r') as f:
    vocab = json.load(f)

In [3]:
in_vocab_size = len(vocab['in_token_to_idx'])
out_vocab_size = len(vocab['out_idx_to_token'])

In [11]:
model = SymbolicOperator(
    in_vocab_size,
    out_vocab_size,
    eos_idx=vocab['out_token_to_idx']['<EOS>'],
    max_program_steps=3,
    gate_activation_train='softmax',
    gate_activation_eval='softmax',
    gate_activation_temperature=1.0,
    read_activation_train='softmax',
    read_activation_eval='argmax',
    write_activation_train='softmax',
    write_activation_eval='argmax',
    use_adaptive_steps=True
)

In [12]:
checkpoint_path = 'checkpoints/0ccd2a7118264657a5b2a5fe16ff7bb9'
model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu')))
model.eval()

SymbolicOperator(
  (gate_attention_activation): AttentionActivation()
  (read_attention_activation): AttentionActivation()
  (write_attention_activation): AttentionActivation()
  (read_attention): Attention(
    (attention_activation): AttentionActivation()
  )
  (gate_embedding): Embedding(16, 2)
  (program_embedding): Embedding(16, 200)
  (primitive_embedding): Embedding(16, 8)
  (keep_going_linear): Linear(in_features=8, out_features=1, bias=True)
  (executor_rnn_cell): GRUCell(200, 384)
)

In [24]:
def eval_text(instruction_text, action_text):
    instruction_words = instruction_text.split()
    action_words = action_text.split()
    
    instruction = []
    for i, token in enumerate(instruction_words):
        idx = int(vocab['in_token_to_idx'][token])
        idx = torch.tensor(idx)
        instruction.append(idx)

    action = []
    for i, token in enumerate(action_words):
        idx = int(vocab['out_token_to_idx'][token])
        idx = torch.tensor(idx)
        action.append(idx)

    instructions = torch.tensor([instruction])
    actions = torch.tensor([action])

    with torch.no_grad():
        output, true_actions, keep_going_loss = model(instructions, actions)
        
    return output[0], true_actions[0]

In [25]:
instruction_text = '<SOS> right turn <EOS>'
action_text = '<SOS> I_TURN_RIGHT <EOS>'

output, true_actions = eval_text(instruction_text, action_text)

print(true_actions)
print(output.argmax(0))

tensor([1, 3])
tensor([1, 1])


In [42]:
def eval_dataset(dataset):
    correct_examples = []
    incorrect_examples = []
    pbar = tqdm(dataset)
    for instruction_tensor, action_tensor, instruction_words, action_words in pbar:
        instruction_text = ' '.join(instruction_words)
        action_text = ' '.join(action_words)

        if len(action_text.split()) <= max_output_len + 2:
            output, true_actions = eval_text(instruction_text, action_text)

            if (output.argmax(0) == true_actions).all().item():
                correct_examples.append([instruction_text, action_text])
            else:
                incorrect_examples.append([instruction_text, action_text])

            pbar.set_description('Acc: {}'.format(len(correct_examples) / (len(correct_examples) + len(incorrect_examples))))
            
    return correct_examples, incorrect_examples

In [43]:
from data import ScanDataset

train_path = 'data/SCAN/tasks_train_addprim_jump_transcoded.txt'
test_path = 'data/SCAN/tasks_test_addprim_jump_transcoded.txt'

max_output_len = 6

def output_filter(action):
    return len(action) <= max_output_len

test_dataset = ScanDataset(test_path, vocab, out_filter_fn=output_filter)
test_correct_examples, test_incorrect_examples = eval_dataset(test_dataset)

train_dataset = ScanDataset(train_path, vocab, out_filter_fn=output_filter)
train_correct_examples, train_incorrect_examples = eval_dataset(train_dataset)






  0%|          | 0/1258 [00:00<?, ?it/s]




Acc: 1.0:   0%|          | 0/1258 [00:00<?, ?it/s]




Acc: 1.0:   0%|          | 0/1258 [00:00<?, ?it/s]




Acc: 1.0:   0%|          | 2/1258 [00:00<01:07, 18.55it/s]




Acc: 1.0:   0%|          | 2/1258 [00:00<01:07, 18.55it/s]




Acc: 1.0:   0%|          | 2/1258 [00:00<01:07, 18.55it/s]




Acc: 1.0:   0%|          | 4/1258 [00:00<01:06, 18.82it/s]




Acc: 1.0:   0%|          | 4/1258 [00:00<01:06, 18.82it/s]




Acc: 1.0:   0%|          | 4/1258 [00:00<01:06, 18.82it/s]




Acc: 1.0:   0%|          | 6/1258 [00:00<01:06, 18.74it/s]




Acc: 1.0:   0%|          | 6/1258 [00:00<01:06, 18.74it/s]




Acc: 1.0:   0%|          | 6/1258 [00:00<01:06, 18.74it/s]




Acc: 1.0:   1%|          | 8/1258 [00:00<01:06, 18.82it/s]




Acc: 1.0:   1%|          | 8/1258 [00:00<01:06, 18.82it/s]




Acc: 1.0:   1%|          | 8/1258 [00:00<01:06, 18.82it/s]




Acc: 1.0:   1%|          | 8/1258 [00:00<01:06, 18.82it/s]




Acc: 1.0:   1%|      

Acc: 0.9552238805970149:   5%|▌         | 65/1258 [00:03<01:05, 18.32it/s]




Acc: 0.9558823529411765:   5%|▌         | 65/1258 [00:03<01:05, 18.32it/s]




Acc: 0.9558823529411765:   5%|▌         | 68/1258 [00:03<01:01, 19.41it/s]




Acc: 0.9565217391304348:   5%|▌         | 68/1258 [00:03<01:01, 19.41it/s]




Acc: 0.9571428571428572:   5%|▌         | 68/1258 [00:03<01:01, 19.41it/s]




Acc: 0.9577464788732394:   5%|▌         | 68/1258 [00:03<01:01, 19.41it/s]




Acc: 0.9577464788732394:   6%|▌         | 71/1258 [00:03<01:01, 19.41it/s]




Acc: 0.9583333333333334:   6%|▌         | 71/1258 [00:03<01:01, 19.41it/s]




Acc: 0.958904109589041:   6%|▌         | 71/1258 [00:03<01:01, 19.41it/s] 




Acc: 0.958904109589041:   6%|▌         | 73/1258 [00:03<01:00, 19.48it/s]




Acc: 0.9594594594594594:   6%|▌         | 73/1258 [00:03<01:00, 19.48it/s]




Acc: 0.96:   6%|▌         | 73/1258 [00:03<01:00, 19.48it/s]              




Acc: 0.9605263157894737:   6%|▌         | 73/1258 [00

Acc: 0.9538461538461539:  10%|█         | 127/1258 [00:06<00:59, 19.09it/s]




Acc: 0.9538461538461539:  10%|█         | 130/1258 [00:06<00:58, 19.43it/s]




Acc: 0.9541984732824428:  10%|█         | 130/1258 [00:06<00:58, 19.43it/s]




Acc: 0.9545454545454546:  10%|█         | 130/1258 [00:06<00:58, 19.43it/s]




Acc: 0.9545454545454546:  10%|█         | 132/1258 [00:06<00:57, 19.59it/s]




Acc: 0.9548872180451128:  10%|█         | 132/1258 [00:06<00:57, 19.59it/s]




Acc: 0.9552238805970149:  10%|█         | 132/1258 [00:06<00:57, 19.59it/s]




Acc: 0.9552238805970149:  11%|█         | 134/1258 [00:06<00:57, 19.52it/s]




Acc: 0.9555555555555556:  11%|█         | 134/1258 [00:06<00:57, 19.52it/s]




Acc: 0.9485294117647058:  11%|█         | 134/1258 [00:06<00:57, 19.52it/s]




Acc: 0.9485294117647058:  11%|█         | 136/1258 [00:06<00:57, 19.48it/s]




Acc: 0.948905109489051:  11%|█         | 136/1258 [00:06<00:57, 19.48it/s] 




Acc: 0.9492753623188406:  11%|█         

Acc: 0.9222797927461139:  15%|█▌        | 191/1258 [00:09<00:51, 20.90it/s]




Acc: 0.9175257731958762:  15%|█▌        | 191/1258 [00:09<00:51, 20.90it/s]




Acc: 0.9175257731958762:  15%|█▌        | 194/1258 [00:09<00:51, 20.60it/s]




Acc: 0.9179487179487179:  15%|█▌        | 194/1258 [00:09<00:51, 20.60it/s]




Acc: 0.9183673469387755:  15%|█▌        | 194/1258 [00:09<00:51, 20.60it/s]




Acc: 0.9137055837563451:  15%|█▌        | 194/1258 [00:09<00:51, 20.60it/s]




Acc: 0.9137055837563451:  16%|█▌        | 197/1258 [00:09<00:50, 20.93it/s]




Acc: 0.9141414141414141:  16%|█▌        | 197/1258 [00:09<00:50, 20.93it/s]




Acc: 0.914572864321608:  16%|█▌        | 197/1258 [00:09<00:50, 20.93it/s] 




Acc: 0.915:  16%|█▌        | 197/1258 [00:09<00:50, 20.93it/s]            




Acc: 0.915:  16%|█▌        | 200/1258 [00:09<00:50, 20.86it/s]




Acc: 0.9154228855721394:  16%|█▌        | 200/1258 [00:10<00:50, 20.86it/s]




Acc: 0.9158415841584159:  16%|█▌        | 200/1258 [00

Acc: 0.914396887159533:  20%|██        | 257/1258 [00:12<00:49, 20.28it/s]




Acc: 0.9147286821705426:  20%|██        | 257/1258 [00:12<00:49, 20.28it/s]




Acc: 0.915057915057915:  20%|██        | 257/1258 [00:12<00:49, 20.28it/s] 




Acc: 0.9153846153846154:  20%|██        | 257/1258 [00:12<00:49, 20.28it/s]




Acc: 0.9153846153846154:  21%|██        | 260/1258 [00:12<00:47, 21.17it/s]




Acc: 0.9157088122605364:  21%|██        | 260/1258 [00:12<00:47, 21.17it/s]




Acc: 0.916030534351145:  21%|██        | 260/1258 [00:12<00:47, 21.17it/s] 




Acc: 0.9163498098859315:  21%|██        | 260/1258 [00:13<00:47, 21.17it/s]




Acc: 0.9163498098859315:  21%|██        | 263/1258 [00:13<00:49, 20.29it/s]




Acc: 0.9128787878787878:  21%|██        | 263/1258 [00:13<00:49, 20.29it/s]




Acc: 0.9132075471698113:  21%|██        | 263/1258 [00:13<00:49, 20.29it/s]




Acc: 0.9135338345864662:  21%|██        | 263/1258 [00:13<00:49, 20.29it/s]




Acc: 0.9135338345864662:  21%|██        |

...

Acc: 0.9528405956977386:  92%|█████████▏| 3623/3922 [02:15<00:11, 25.39it/s]




Acc: 0.9528405956977386:  92%|█████████▏| 3626/3922 [02:15<00:12, 23.70it/s]




Acc: 0.9528535980148883:  92%|█████████▏| 3626/3922 [02:15<00:12, 23.70it/s]




Acc: 0.9528665931642778:  92%|█████████▏| 3626/3922 [02:15<00:12, 23.70it/s]




Acc: 0.9528795811518325:  92%|█████████▏| 3626/3922 [02:15<00:12, 23.70it/s]




Acc: 0.9528795811518325:  93%|█████████▎| 3629/3922 [02:15<00:12, 22.88it/s]




Acc: 0.952892561983471:  93%|█████████▎| 3629/3922 [02:15<00:12, 22.88it/s] 




Acc: 0.952905535665106:  93%|█████████▎| 3629/3922 [02:15<00:12, 22.88it/s]




Acc: 0.9529185022026432:  93%|█████████▎| 3629/3922 [02:15<00:12, 22.88it/s]




Acc: 0.9529185022026432:  93%|█████████▎| 3632/3922 [02:15<00:12, 22.87it/s]




Acc: 0.9529314616019818:  93%|█████████▎| 3632/3922 [02:15<00:12, 22.87it/s]




Acc: 0.9529444138690149:  93%|█████████▎| 3632/3922 [02:15<00:12, 22.87it/s]




Acc: 0.9529573590096286:  93%

Acc: 0.9528583039826605:  94%|█████████▍| 3691/3922 [02:17<00:08, 26.23it/s]




Acc: 0.9528710725893824:  94%|█████████▍| 3691/3922 [02:17<00:08, 26.23it/s]




Acc: 0.9528838342810723:  94%|█████████▍| 3691/3922 [02:17<00:08, 26.23it/s]




Acc: 0.952896589063346:  94%|█████████▍| 3691/3922 [02:18<00:08, 26.23it/s] 




Acc: 0.952896589063346:  94%|█████████▍| 3694/3922 [02:18<00:08, 26.25it/s]




Acc: 0.9529093369418132:  94%|█████████▍| 3694/3922 [02:18<00:08, 26.25it/s]




Acc: 0.952922077922078:  94%|█████████▍| 3694/3922 [02:18<00:08, 26.25it/s] 




Acc: 0.9529348120097376:  94%|█████████▍| 3694/3922 [02:18<00:08, 26.25it/s]




Acc: 0.9529348120097376:  94%|█████████▍| 3697/3922 [02:18<00:09, 22.92it/s]




Acc: 0.952947539210384:  94%|█████████▍| 3697/3922 [02:18<00:09, 22.92it/s] 




Acc: 0.9529602595296026:  94%|█████████▍| 3697/3922 [02:18<00:09, 22.92it/s]




Acc: 0.952972972972973:  94%|█████████▍| 3697/3922 [02:18<00:09, 22.92it/s] 




Acc: 0.952972972972973:  94%|

Acc: 0.9536987759446514:  96%|█████████▌| 3756/3922 [02:20<00:06, 25.84it/s]




Acc: 0.9537110933758979:  96%|█████████▌| 3756/3922 [02:20<00:06, 25.84it/s]




Acc: 0.9537110933758979:  96%|█████████▌| 3759/3922 [02:20<00:06, 25.08it/s]




Acc: 0.9537234042553191:  96%|█████████▌| 3759/3922 [02:20<00:06, 25.08it/s]




Acc: 0.9537357085881415:  96%|█████████▌| 3759/3922 [02:20<00:06, 25.08it/s]




Acc: 0.9537480063795853:  96%|█████████▌| 3759/3922 [02:20<00:06, 25.08it/s]




Acc: 0.9537480063795853:  96%|█████████▌| 3762/3922 [02:20<00:06, 25.69it/s]




Acc: 0.9537602976348658:  96%|█████████▌| 3762/3922 [02:20<00:06, 25.69it/s]




Acc: 0.9537725823591924:  96%|█████████▌| 3762/3922 [02:20<00:06, 25.69it/s]




Acc: 0.9537848605577689:  96%|█████████▌| 3762/3922 [02:20<00:06, 25.69it/s]




Acc: 0.9537848605577689:  96%|█████████▌| 3765/3922 [02:20<00:05, 26.44it/s]




Acc: 0.9535315985130112:  96%|█████████▌| 3765/3922 [02:20<00:05, 26.44it/s]




Acc: 0.9535439341651182:  96

Acc: 0.9536891679748822:  97%|█████████▋| 3820/3922 [02:23<00:04, 20.56it/s]




Acc: 0.9537012817159299:  97%|█████████▋| 3820/3922 [02:23<00:04, 20.56it/s]




Acc: 0.9537012817159299:  97%|█████████▋| 3823/3922 [02:23<00:05, 19.06it/s]




Acc: 0.953713389121339:  97%|█████████▋| 3823/3922 [02:23<00:05, 19.06it/s] 




Acc: 0.9537254901960784:  97%|█████████▋| 3823/3922 [02:23<00:05, 19.06it/s]




Acc: 0.9537375849451124:  97%|█████████▋| 3823/3922 [02:23<00:05, 19.06it/s]




Acc: 0.9537375849451124:  98%|█████████▊| 3826/3922 [02:23<00:04, 20.93it/s]




Acc: 0.9537496733733996:  98%|█████████▊| 3826/3922 [02:23<00:04, 20.93it/s]




Acc: 0.9537617554858934:  98%|█████████▊| 3826/3922 [02:23<00:04, 20.93it/s]




Acc: 0.9537738312875425:  98%|█████████▊| 3826/3922 [02:23<00:04, 20.93it/s]




Acc: 0.9537859007832898:  98%|█████████▊| 3826/3922 [02:23<00:04, 20.93it/s]




Acc: 0.9537859007832898:  98%|█████████▊| 3830/3922 [02:23<00:04, 22.98it/s]




Acc: 0.9537979639780736:  98

Acc: 0.9542063287882686:  99%|█████████▉| 3884/3922 [02:26<00:01, 25.57it/s]




Acc: 0.9542063287882686:  99%|█████████▉| 3887/3922 [02:26<00:01, 24.19it/s]




Acc: 0.9542181069958847:  99%|█████████▉| 3887/3922 [02:26<00:01, 24.19it/s]




Acc: 0.9542298791463101:  99%|█████████▉| 3887/3922 [02:26<00:01, 24.19it/s]




Acc: 0.954241645244216:  99%|█████████▉| 3887/3922 [02:26<00:01, 24.19it/s] 




Acc: 0.954241645244216:  99%|█████████▉| 3890/3922 [02:26<00:01, 23.48it/s]




Acc: 0.9542534052942688:  99%|█████████▉| 3890/3922 [02:26<00:01, 23.48it/s]




Acc: 0.9540082219938335:  99%|█████████▉| 3890/3922 [02:26<00:01, 23.48it/s]




Acc: 0.954020035961983:  99%|█████████▉| 3890/3922 [02:26<00:01, 23.48it/s] 




Acc: 0.954020035961983:  99%|█████████▉| 3893/3922 [02:26<00:01, 20.79it/s]




Acc: 0.9540318438623523:  99%|█████████▉| 3893/3922 [02:26<00:01, 20.79it/s]




Acc: 0.9540436456996149:  99%|█████████▉| 3893/3922 [02:26<00:01, 20.79it/s]




Acc: 0.9540554414784395:  99%|

In [48]:
incorrect_only_in_test = []
train_incorrect_instructions = list(zip(*train_incorrect_examples))[0]
for instruction, action in test_incorrect_examples:
    if instruction.replace('jump', 'walk') not in train_incorrect_instructions:
        incorrect_only_in_test.append([instruction, action])

In [18]:
scratch_history = []
for word in model.scratch_history:
    scratch_history.append([])
    for step in word:
        gate = step[0][0][0].long()
        keep_going_gate = step[1][0][0].item()
        read = step[2][0][0].long()
        write = step[3][0][0].long()
        scratch = step[4].argmax(-1)[0]
        scratch_history[-1].append([gate, keep_going_gate, read, write, scratch])

for i, steps in enumerate(scratch_history):
    word = instruction_text[i+1]
    print('seeing', word)
    for gate, keep_going_gate, read, write, scratch_pad in steps:
        tokens = []
        for token_idx in scratch_pad:
            token = vocab['out_idx_to_token'][str(token_idx.item())]
            tokens.append(token)
        print('keep going:', keep_going_gate)
        if gate[0] == 1:
            print('read:', word)
        else:
            print('read:', read)
        print('write:', write)
        print(tokens)
    print()

seeing right
keep going: 1.0
read: right
write: tensor([1, 0])
['I_TURN_RIGHT', '<EOS>']
keep going: 1.0
read: right
write: tensor([1, 0])
['I_TURN_RIGHT', '<EOS>']
keep going: 0.0
read: right
write: tensor([0, 0])
['I_TURN_RIGHT', 'I_TURN_RIGHT']

seeing turn
keep going: 1.0
read: turn
write: tensor([0, 1])
['I_TURN_RIGHT', 'I_TURN_RIGHT']
keep going: 0.0
read: turn
write: tensor([0, 0])
['I_TURN_RIGHT', 'I_TURN_RIGHT']
keep going: 0.0
read: turn
write: tensor([0, 0])
['I_TURN_RIGHT', 'I_TURN_RIGHT']



In [16]:
model.scratch_history[0]

[[tensor([[[1., 0.]]]),
  tensor([[1.]]),
  tensor([[[1., 0., 0.]]]),
  tensor([[[1., 0., 0.]]]),
  tensor([[[-4.1023,  4.8622, -4.5779, -1.9348, -1.6738, -3.7650, -1.9237,
            -2.2320],
           [ 0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
             0.0000],
           [ 0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
             0.0000]]])],
 [tensor([[[1., 0.]]]),
  tensor([[1.]]),
  tensor([[[1., 0., 0.]]]),
  tensor([[[1., 0., 0.]]]),
  tensor([[[-4.7281,  5.6038, -5.2762, -2.3825, -1.9291, -4.3393, -2.2171,
            -2.5725],
           [ 0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
             0.0000],
           [ 0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
             0.0000]]])],
 [tensor([[[1., 0.]]]),
  tensor([[0.]]),
  tensor([[[1., 0., 0.]]]),
  tensor([[[0., 0., 0.]]]),
  tensor([[[-4.7281,  5.6038, -5.2762, -2.3825, -1.9291, -4.3393, -2.2171,
            -2.5725],
           [-1.8496